In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"techworldzeeshan","key":"b13b8d5f22898935cf67d2053d35f67f"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
# Download the dataset
!kaggle datasets download -d chethuhn/Network-Intrusion-dataset

# Create a directory to unzip the dataset
!mkdir -p CIC-IDS_dataset

# Unzip the dataset
!unzip Network-Intrusion-dataset.zip -d CIC-IDS_dataset


Dataset URL: https://www.kaggle.com/datasets/chethuhn/Network-Intrusion-dataset
License(s): CC0-1.0
 91% 210M/230M [00:02<00:00, 72.3MB/s]
100% 230M/230M [00:02<00:00, 105MB/s] 
Archive:  Network-Intrusion-dataset.zip
  inflating: CIC-IDS_dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv  
  inflating: CIC-IDS_dataset/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv  
  inflating: CIC-IDS_dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv  
  inflating: CIC-IDS_dataset/Monday-WorkingHours.pcap_ISCX.csv  
  inflating: CIC-IDS_dataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: CIC-IDS_dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: CIC-IDS_dataset/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: CIC-IDS_dataset/Wednesday-workingHours.pcap_ISCX.csv  


In [4]:
import os

print("Listing files in 'CIC-IDS_dataset' directory after extraction:")
for root, dirs, files in os.walk('CIC-IDS_dataset'):
    for filename in files:
        print(f"Found file: {os.path.join(root, filename)}")


Listing files in 'CIC-IDS_dataset' directory after extraction:
Found file: CIC-IDS_dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv
Found file: CIC-IDS_dataset/Tuesday-WorkingHours.pcap_ISCX.csv
Found file: CIC-IDS_dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Found file: CIC-IDS_dataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Found file: CIC-IDS_dataset/Monday-WorkingHours.pcap_ISCX.csv
Found file: CIC-IDS_dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Found file: CIC-IDS_dataset/Wednesday-workingHours.pcap_ISCX.csv
Found file: CIC-IDS_dataset/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv


In [5]:
import pandas as pd

def load_data_from_folder(folder_path, file_extension='.csv'):  # Adjust extension as needed
    data_frames = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(file_extension):  # Adjust the extension as needed
                file_path = os.path.join(root, file)
                print(f"Loading {file_path}")
                df = pd.read_csv(file_path)  # Adjust parameters as needed
                data_frames.append(df)
    if not data_frames:
        raise ValueError(f"No {file_extension} files found in the specified folder.")
    return pd.concat(data_frames, ignore_index=True)

# Specify the folder containing the dataset
dataset_folder = 'CIC-IDS_dataset'
df = load_data_from_folder(dataset_folder)

# Verify the loaded DataFrame
df.head()

# Proceed with preprocessing
X = df.iloc[:, :-1]
y = df.iloc[:, -1]


Loading CIC-IDS_dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv
Loading CIC-IDS_dataset/Tuesday-WorkingHours.pcap_ISCX.csv
Loading CIC-IDS_dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Loading CIC-IDS_dataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Loading CIC-IDS_dataset/Monday-WorkingHours.pcap_ISCX.csv
Loading CIC-IDS_dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Loading CIC-IDS_dataset/Wednesday-workingHours.pcap_ISCX.csv
Loading CIC-IDS_dataset/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv


In [6]:
import numpy as np

# Replace inf values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Check for NaN values
print("Number of NaN values in each column:\n", X.isna().sum())

# Option 1: Drop rows with NaN values
# X.dropna(inplace=True)

# Option 2: Fill NaN values with a specific value (e.g., mean of the column)
X.fillna(X.mean(), inplace=True)


Number of NaN values in each column:
  Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
 Active Min                    0
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
Length: 78, dtype: int64


In [7]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Encoding the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Building a simple neural network
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.2)

Epoch 1/3
56615/56615 [==============================] - 112s 2ms/step - loss: 0.0643 - accuracy: 0.9741 - val_loss: 0.0511 - val_accuracy: 0.9787
Epoch 2/3
56615/56615 [==============================] - 108s 2ms/step - loss: 0.0464 - accuracy: 0.9809 - val_loss: 0.0416 - val_accuracy: 0.9836
Epoch 3/3
56615/56615 [==============================] - 107s 2ms/step - loss: 0.0364 - accuracy: 0.9867 - val_loss: 0.0433 - val_accuracy: 0.9882


In [9]:

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

17693/17693 [==============================] - 23s 1ms/step - loss: 0.0439 - accuracy: 0.9883
Test accuracy: 0.9882592558860779


In [10]:


import pickle

# Save the LabelEncoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Save the StandardScaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save the model
model.save('intrusion_detection_model.h5')

print("Label encoder, scaler, and model saved successfully!")


Label encoder, scaler, and model saved successfully!


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
from google.colab import files
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle

# Load the saved model, scaler, and label encoder
loaded_model = tf.keras.models.load_model('intrusion_detection_model.h5')
with open('scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)
with open('label_encoder.pkl', 'rb') as f:
    loaded_label_encoder = pickle.load(f)

# Define the feature names (replace with your actual feature names)
feature_names = [
    "Destination Port", "Flow Duration", "Total Fwd Packets", "Total Backward Packets",
    "Total Length of Fwd Packets", "Total Length of Bwd Packets", "Fwd Packet Length Max",
    "Fwd Packet Length Min", "Fwd Packet Length Mean", "Fwd Packet Length Std",
    "Bwd Packet Length Max", "Bwd Packet Length Min", "Bwd Packet Length Mean",
    "Bwd Packet Length Std", "Flow Bytes/s", "Flow Packets/s", "Flow IAT Mean",
    "Flow IAT Std", "Flow IAT Max", "Flow IAT Min", "Fwd IAT Total", "Fwd IAT Mean",
    "Fwd IAT Std", "Fwd IAT Max", "Fwd IAT Min", "Bwd IAT Total", "Bwd IAT Mean",
    "Bwd IAT Std", "Bwd IAT Max", "Bwd IAT Min", "Fwd PSH Flags", "Bwd PSH Flags",
    "Fwd URG Flags", "Bwd URG Flags", "Fwd Header Length", "Bwd Header Length",
    "Fwd Packets/s", "Bwd Packets/s", "Min Packet Length", "Max Packet Length",
    "Packet Length Mean", "Packet Length Std", "Packet Length Variance",
    "FIN Flag Count", "SYN Flag Count", "RST Flag Count", "PSH Flag Count",
    "ACK Flag Count", "URG Flag Count", "CWE Flag Count", "ECE Flag Count",
    "Down/Up Ratio", "Average Packet Size", "Avg Fwd Segment Size", "Avg Bwd Segment Size",
    "Fwd Header Length.1", "Fwd Avg Bytes/Bulk", "Fwd Avg Packets/Bulk",
    "Fwd Avg Bulk Rate", "Bwd Avg Bytes/Bulk", "Bwd Avg Packets/Bulk",
    "Bwd Avg Bulk Rate", "Subflow Fwd Packets", "Subflow Fwd Bytes",
    "Subflow Bwd Packets", "Subflow Bwd Bytes", "Init_Win_bytes_forward",
    "Init_Win_bytes_backward", "act_data_pkt_fwd", "min_seg_size_forward",
    "Active Mean", "Active Std", "Active Max", "Active Min", "Idle Mean",
    "Idle Std", "Idle Max", "Idle Min"
]

# Function to get user input and make a prediction
def predict_intrusion():
    """Gets user input for features and predicts the intrusion type."""

    # Get user input for features
    features = []
    for i, feature_name in enumerate(feature_names):
        feature = float(input(f"Enter value for {feature_name}: "))
        features.append(feature)

    # Create a numpy array with the input values, reshaped for a single sample
    input_data = np.array([features])

    # Scale the input data using the loaded scaler
    scaled_input = loaded_scaler.transform(input_data)

    # Make a prediction using the loaded model
    prediction = loaded_model.predict(scaled_input)

    # Get the predicted class index
    predicted_class_index = np.argmax(prediction)

    # Convert the predicted class index to the original label
    predicted_label = loaded_label_encoder.classes_[predicted_class_index]

    # Print the predicted label
    print(f"Predicted Intrusion Type: {predicted_label}")

# Call the function to start the prediction process
predict_intrusion()


Enter value for Destination Port: 54865
Enter value for Flow Duration: 3
Enter value for Total Fwd Packets: 2
Enter value for Total Backward Packets: 0
Enter value for Total Length of Fwd Packets: 12
Enter value for Total Length of Bwd Packets: 0
Enter value for Fwd Packet Length Max: 6
Enter value for Fwd Packet Length Min: 6
Enter value for Fwd Packet Length Mean: 6
Enter value for Fwd Packet Length Std: 0
Enter value for Bwd Packet Length Max: 0
Enter value for Bwd Packet Length Min: 0
Enter value for Bwd Packet Length Mean: 0
Enter value for Bwd Packet Length Std: 0
Enter value for Flow Bytes/s: 4000000
Enter value for Flow Packets/s: 666666.7
Enter value for Flow IAT Mean: 3
Enter value for Flow IAT Std: 0
Enter value for Flow IAT Max: 3
Enter value for Flow IAT Min: 3
Enter value for Fwd IAT Total: 3
Enter value for Fwd IAT Mean: 3
Enter value for Fwd IAT Std: 0
Enter value for Fwd IAT Max: 3
Enter value for Fwd IAT Min: 3
Enter value for Bwd IAT Total: 0
Enter value for Bwd IAT 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
